In [1]:
import pandas as pd
import numpy as np
import csv
import time
from datetime import datetime,timedelta
import datetime

In [2]:
g=open('life_cycle_data.csv',encoding='utf-8')
life_data=pd.read_csv(g)
g.close()

In [3]:
life_data.head()

,company_flag,cust_code,last_order_time,first_diff,last_two_order_time,two_diff
0,AK,36189144,2020/5/17,1,2020/5/6,11
1,AK,36189161,2020/5/17,1,2020/5/14,3
2,AK,36189406,2020/5/17,1,2020/5/15,2
3,AK,36189728,2020/5/17,1,2020/5/9,8
4,AK,36266284,2020/5/17,1,2020/5/14,3


In [4]:
life_data.dtypes

company_flag           object
cust_code               int64
last_order_time        object
first_diff              int64
last_two_order_time    object
two_diff                int64
dtype: object

In [5]:
life_data.isnull().any()

company_flag           False
cust_code              False
last_order_time        False
first_diff             False
last_two_order_time     True
two_diff               False
dtype: bool

In [6]:
cust_flag=life_data['company_flag'].drop_duplicates()

In [7]:
len(cust_flag)

97

In [8]:
def cycle(z):
    a=int(z[3])
    b=int(z[5])
    if a<=5 and b<=5:
        c='高活跃'
    elif (a>5 and a<=14 and b>5 and b<=14)or (a<=5 and b>5 and b<=14) or (a>5 and a<=14 and b<=5):
        c='中活跃'    
    elif (a>14 and a<=30 and b>14 and b<=30) or (a<30 and b>14 and b<=30) or (a>14 and a<=30 and b<30):
        c='低活跃'
    elif a<=30 and b>=30:
        c='潜在活跃'
    elif a>30 and a<=60:
        c='休眠用户'
    elif a>60 and a<=90:
        c='流失用户'
    elif b>90:
        c='转新用户'
    else:
        c='流失用户'
    return c


In [9]:
life_cycle_data=pd.DataFrame(columns=['company_flag','cust_code','last_order_time','first_diff','last_two_order_time','two_diff','cycle ','day_id'])

In [10]:
life_cycle_data.head()

,company_flag,cust_code,last_order_time,first_diff,last_two_order_time,two_diff,cycle,day_id


In [11]:
company_flag1=[]
x1=[]
y1=[]
m1=[]
n1=[]
print("开始计算生命周期")
for i in range(len(cust_flag)):
    #print(i)
    flag_data=life_data[life_data['company_flag']==cust_flag.iloc[i]]
    flag_data=flag_data.sort_values(by=['company_flag','first_diff'],ascending=True)
    flag_data=flag_data.reset_index(drop=True)
    company_flag=cust_flag.iloc[i]
    company_flag1.append(company_flag)
    #取行数
    nl=flag_data.count()
    n=nl.first_diff
    a=np.round(n*0.25)
    x=flag_data.iloc[int(a)].first_diff
    x1.append(x)
    a1=np.round(n*0.5)
    y=flag_data.iloc[int(a1)].first_diff
    y1.append(y)
    a2=np.round(n*0.7)
    try:
        m=flag_data.iloc[int(a2)].first_diff
    except:
        m=0
    m1.append(m)
    a3=np.round(n*0.9)
    try:
        n=flag_data.iloc[int(a3)].first_diff
    except:
        n=0
    n1.append(n)
    #print(x,y,m,n)
    flag_data['cycle']=flag_data.apply(cycle,axis=1)
    day_id=(datetime.datetime.now()+datetime.timedelta(days=-1)).strftime("%Y-%m-%d")
    flag_data['day_id']=day_id
    #print(flag_data)
    life_cycle_data=pd.concat([life_cycle_data,flag_data],axis=0)

开始计算生命周期


In [12]:
life_cycle_data.head()

,company_flag,cust_code,last_order_time,first_diff,last_two_order_time,two_diff,cycle,day_id,cycle
0,AK,36189144,2020/5/17,1,2020/5/6,11,NaN,2021-02-21,中活跃
1,AK,36189161,2020/5/17,1,2020/5/14,3,NaN,2021-02-21,高活跃
2,AK,36189406,2020/5/17,1,2020/5/15,2,NaN,2021-02-21,高活跃
3,AK,36189728,2020/5/17,1,2020/5/9,8,NaN,2021-02-21,中活跃
4,AK,36266284,2020/5/17,1,2020/5/14,3,NaN,2021-02-21,高活跃


In [13]:
len(life_cycle_data)

211026

In [14]:
life_cycle_data['cycle']=life_cycle_data['cycle'].fillna('流失用户')

In [15]:
cyclelife_cycle_data_result=life_cycle_data[['day_id','company_flag','cust_code','cycle']]

In [16]:
cyclelife_cycle_data_result.head()

,day_id,company_flag,cust_code,cycle
0,2021-02-21,AK,36189144,中活跃
1,2021-02-21,AK,36189161,高活跃
2,2021-02-21,AK,36189406,高活跃
3,2021-02-21,AK,36189728,中活跃
4,2021-02-21,AK,36266284,高活跃
